In [143]:
import pandas as pd
from seml import get_results

In [ ]:
df = get_results(
    'dp_timeseries_standard_train_dp_eval',
    [
        'config.seed',
        'config.estimator_name',
        'config.inference_kwargs.subsample_transform_kwargs.subsampling_rate',
        'config.inference_kwargs.neighboring_relation.target_sensitivity',
        'config.inference_kwargs.budget_epsilon',
        'config.inference_kwargs.budget_delta',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MASE'
    ],
    to_data_frame=True,
    filter_dict={
        'config.dataset_kwargs.dataset_name': 'electricity',
        'config.inference_kwargs.imputation_transform_name': 'MeanValueImputation',  # Try both of these
        'config.estimator_kwargs.trainer_kwargs.max_epochs': 8000,
        'config.top_level_mode': 'sampling_without_replacement',
        'config.instances_per_sequence': 1,
        'config.estimator_kwargs.batch_size': 64,
        'config.estimator_kwargs.relative_context_length': 2,
        'config.inference_kwargs.subsample_transform_kwargs.subsampling_rate': {'$in': [0.5, 0.75, 1.0]},
        'config.estimator_kwargs.lags_seq': {'$in': [None, [
            1, 2, 3, 4, 5, 6, 7, 23, 24, 25, 47, 48, 49, 71, 72, 73,
            95, 96, 97, 119, 120, 121, 143, 144, 145, 167, 168, 169]]}
    }
)

In [145]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [ ]:
df

In [147]:
def create_table(df: pd.DataFrame,
                 budget_delta: int,
                 target_sensitivity: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['budget_delta'] == budget_delta)
        & (df['target_sensitivity'] == target_sensitivity)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['estimator_name', 'budget_delta', 'budget_epsilon', 'target_sensitivity', 'subsampling_rate']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['estimator_name', 'budget_delta', 'budget_epsilon', 'target_sensitivity', 'subsampling_rate']
        ).std().reset_index()

    df = df.drop(columns=[
        '_id',
        'seed', 'budget_delta',
        'target_sensitivity', '_id'])

    df = df.sort_values(by=['estimator_name', 'subsampling_rate', 'budget_epsilon'],
                        ascending=[False, False, True])

    return df

In [148]:
budget_delta=0.0001
target_sensitivity=10.0

In [ ]:
df_mean_no_subsampling = create_table(df, budget_delta=budget_delta, target_sensitivity=target_sensitivity)
df_mean_no_subsampling

In [ ]:
df_std_no_subsampling = create_table(df, budget_delta=budget_delta, target_sensitivity=target_sensitivity)
df_std_no_subsampling

In [ ]:
def rate(i):
    mod = i % 3

    if mod == 0:
        return 1.0
    elif mod == 1:
        return 0.75
    elif mod == 2:
        return 0.5

i = 0
names = ['SimpleFF', 'DeepAR', 'DLinear']
names = [[x, x, x] for x in names]
names = [x for y in names for x in y]
names = [f'{x} ($r=' + f'{rate(i)}' + '$)' 
        for i, x in enumerate(names)]

acc = f'{names[0]} &'

for i, (mean, std) in enumerate(zip(df_mean_no_subsampling['mean_wQuantileLoss'], df_std_no_subsampling['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 5)  == 0):
        acc += ' ? \\\\'
        print(acc)
        acc = f'{names[i // 5]} &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' ? \\\\ '
print(acc)